In [4]:
# Ambulance Voice Assistant
# This script listens to a user's voice, sends the transcribed text to the Gemini API,
# and expects a summary and a structured data output based on a predefined function.
# Version 2.0: Now includes chat history for conversational context and better error handling.

# --- 1. Installation ---
# In a Google Colab cell or your terminal, run these commands first:
# !pip install google-generativeai speechrecognition gTTS playsound PyAudio
# Note: In some environments, you might need to install portaudio for PyAudio (`sudo apt-get install portaudio19-dev`)

import google.generativeai as genai
import speech_recognition as sr
from gtts import gTTS
import os
import json
from IPython.display import Audio, display
import time

# --- 2. Configuration ---
# IMPORTANT: Add your Gemini API Key here.
# You can get one from Google AI Studio.
# In Google Colab, it's recommended to use "Secrets" to store your key.
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except (ImportError, KeyError):
    # This will prompt you to enter the key if it's not found in Colab secrets
    import getpass
    try:
        GOOGLE_API_KEY = getpass.getpass("Please enter your Google API Key: ")
    except Exception as e:
        print(f"Could not read API key: {e}")
        GOOGLE_API_KEY = None

if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
else:
    print("API Key not provided. The script cannot proceed.")
    # Exit if no API key is available
    exit()


# --- 3. Prompts and Function Schema (from your Canvas) ---

# System Instructions in English and Arabic
system_instructions = """
You are an advanced AI assistant for an ambulance dispatch center. Your task is to accurately and efficiently analyze incoming reports from paramedics.
You must follow these steps in strict order upon receiving any report:
1. First - Create a Summary: Start by writing the heading "Case Summary:". Then, formulate a brief and clear summary of the report in one or two sentences at most, covering the main points of the case.
2. Second - Fill the Report: Immediately after writing the summary, call the function `تعبئة_تقرير_حالة_اسعافية` and use the information from the conversation to fill in all required and optional fields in the report as accurately as possible.
Do not provide any other information or text outside of this specified order.
"""

# The function schema for the ambulance report
ambulance_report_function = {
    "name": "تعبئة_تقرير_حالة_اسعافية",
    "description": "يستخدم لاستخلاص بيانات تقرير حالة إسعافية من محادثة مع المسعف أو من غرفة العمليات وتعبئتها في هيكل منظم.",
    "parameters": {
        "type": "OBJECT",
        "properties": {
            "معلومات_البلاغ": {
                "type": "OBJECT",
                "description": "معلومات وتوقيتات البلاغ الإسعافي.",
                "properties": {
                    "تاريخ_البلاغ": {"type": "STRING", "description": "تاريخ الحالة بالتقويم الهجري أو الميلادي."},
                    "رقم_البلاغ": {"type": "STRING", "description": "الرقم المرجعي للبلاغ."},
                    "زمن_البلاغ": {"type": "STRING", "description": "الوقت الذي تم فيه إنشاء البلاغ."},
                    "موقع_الحالة": {"type": "STRING", "description": "وصف دقيق لموقع الحالة."}
                }
            },
            "بيانات_المريض": {
                "type": "OBJECT", "description": "المعلومات الشخصية للمريض.",
                "properties": {
                    "اسم_المريض": {"type": "STRING", "description": "الاسم الكامل للمريض، أو 'مجهول' إذا لم يكن معروفاً."},
                    "العمر": {"type": "INTEGER", "description": "عمر المريض بالسنوات."},
                    "الجنس": {"type": "STRING", "description": "جنس المريض.", "enum": ["ذكر", "أنثى"]},
                    "الجنسية": {"type": "STRING", "description": "جنسية المريض."},
                    "حالة_الوعي": {"type": "STRING", "description": "حالة وعي المريض عند الوصول.", "enum": ["واعي", "غير واعي"]}
                },
                "required": ["اسم_المريض", "العمر", "الجنس", "حالة_الوعي"]
            },
            "الشكوى_والفحص": {
                "type": "OBJECT", "description": "شكوى المريض الرئيسية ونتائج الفحص السريري.",
                "properties": {
                    "شكوى_المريض_الرئيسية": {"type": "STRING", "description": "الشكوى الأساسية التي أبلغ عنها المريض أو الحاضرون."},
                    "المؤشرات_الحيوية": {
                        "type": "OBJECT", "description": "القياسات الحيوية للمريض.",
                        "properties": {
                            "النبض": {"type": "INTEGER", "description": "معدل نبضات القلب في الدقيقة."},
                            "ضغط_الدم": {"type": "STRING", "description": "قراءة ضغط الدم (مثال: '120/80')."},
                            "معدل_التنفس": {"type": "INTEGER", "description": "عدد الأنفاس في الدقيقة."},
                            "تشبع_الاكسجين": {"type": "INTEGER", "description": "نسبة تشبع الأكسجين في الدم (SpO2)."},
                            "الحرارة": {"type": "NUMBER", "description": "درجة حرارة الجسم المئوية."},
                            "سكر_الدم": {"type": "INTEGER", "description": "مستوى السكر في الدم."},
                            "مقياس_جلاسكو_للغيبوبة": {"type": "INTEGER", "description": "مجموع نقاط مقياس جلاسكو للغيبوبة (GCS)."}
                        }
                    },
                    "التشخيص_المبدئي": {
                        "type": "ARRAY", "description": "قائمة بحالات التشخيص المبدئية بناءً على الفحص.",
                        "items": {"type": "STRING", "enum": ["حالة قلبية", "حالة تنفسية", "حالة عصبية/نفسية", "حالة هضمية", "ضعف عام", "حالة سكرية", "إغماء", "حالة نسائية", "ارتفاع حرارة", "إصابة جنائية", "حادث سير", "سقوط", "كارثة طبيعية", "ازدحام", "انهيار", "انزلاق", "حريق", "احتجاز", "مشاجرة", "إصابة عمل", "غرق"]}
                    },
                    "حدة_الحالة": {"type": "STRING", "description": "تقييم حدة وخطورة الحالة.", "enum": ["مستقرة", "متوسطة", "خطرة", "حالة طارئة", "متوفى"]},
                    "الكشف_السريري_والتاريخ_المرضي": {"type": "STRING", "description": "ملاحظات إضافية من الكشف السريري والتاريخ المرضي للمريض."}
                },
                "required": ["شكوى_المريض_الرئيسية", "حدة_الحالة"]
            },
            "النقل_والاجراءات": {
                "type": "OBJECT", "description": "معلومات عن نقل المريض والإجراءات المتخذة.",
                "properties": {
                    "حالة_النقل": {"type": "STRING", "description": "هل تم نقل المريض أم لا، والسبب.", "enum": ["تم النقل إلى مستشفى", "تم النقل إلى مركز صحي", "رفض النقل", "حالة غير اسعافية", "متوفى في الموقع"]},
                    "اسم_المستشفى_او_المركز": {"type": "STRING", "description": "اسم المستشفى أو المركز الصحي الذي تم نقل المريض إليه."},
                    "ملاحظات_اضافية": {"type": "STRING", "description": "أي ملاحظات إضافية حول الحالة أو الإجراءات."}
                },
                "required": ["حالة_النقل"]
            }
        }
    }
}


# --- 4. Model and Core Functions ---

def speak(text, lang='ar'):
    """Converts text to speech and plays it."""
    try:
        tts = gTTS(text=text, lang=lang, slow=False)
        filename = "response.mp3"
        tts.save(filename)
        print(f"Assistant: {text}")
        display(Audio(filename, autoplay=True))
        time.sleep(len(text.split()) * 0.4 + 0.5) # Adjusted sleep time
        os.remove(filename)
    except Exception as e:
        print(f"Error in text-to-speech: {e}")

def listen_and_transcribe():
    """Listens for audio via microphone and transcribes it."""
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("\n🎙️ Listening... Please state the report.")
        r.pause_threshold = 1.0
        r.adjust_for_ambient_noise(source, duration=1)
        try:
            audio = r.listen(source, timeout=5, phrase_time_limit=15)
            text = r.recognize_google(audio, language='ar-SA')
            print(f"You said: {text}")
            return text
        except sr.UnknownValueError:
            speak("عفواً، لم أستطع فهم ما قلت. هل يمكنك الإعادة؟", lang='ar')
            return None
        except sr.RequestError as e:
            speak(f"Could not request results; {e}", lang='en')
            return None
        except sr.WaitTimeoutError:
            print("Listening timed out while waiting for phrase to start")
            return None

def process_report():
    """Main function to run the assistant loop with chat context."""
    # Initialize the Gemini Model
    model = genai.GenerativeModel(
        model_name='gemini-1.5-flash',
        system_instruction=system_instructions,
        tools=[ambulance_report_function]
    )

    # *** KEY CHANGE: Start a chat session to maintain conversation history ***
    chat = model.start_chat()

    speak("أنا جاهز لاستلام البلاغ.", lang='ar')

    while True:
        transcribed_text = listen_and_transcribe()

        if transcribed_text:
            if any(word in transcribed_text for word in ["توقف", "stop"]):
                 speak("إيقاف المساعد. شكراً لك.", lang='ar')
                 break

            print("\nProcessing with Gemini...")
            try:
                # *** KEY CHANGE: Send message as part of the ongoing chat session ***
                response = chat.send_message(transcribed_text)

                # --- Parse Gemini's Response ---
                response_part = response.parts[0]

                # Check for a text response (the summary or a question)
                if response_part.text:
                    print("\n--- Model's Text Response ---")
                    print(response_part.text)
                    # Speak the text response only if it's a question for the user
                    if not response_part.function_call:
                        speak(response_part.text, lang='ar')

                # Check for a function call (the filled form)
                if response_part.function_call:
                    function_call = response_part.function_call
                    function_name = function_call.name
                    function_args = function_call.args

                    print(f"\n--- Function Call: {function_name} ---")
                    # Pretty print the JSON data
                    print(json.dumps(dict(function_args), indent=2, ensure_ascii=False))
                    speak("تم تسجيل البيانات بنجاح. هل هناك بلاغ آخر؟", lang='ar')

            except Exception as e:
                # *** KEY CHANGE: Added error handling for API calls ***
                print(f"An error occurred while calling the Gemini API: {e}")
                speak("حدث خطأ أثناء معالجة الطلب. يرجى المحاولة مرة أخرى.", lang='ar')

        else:
            print("No valid input received, listening again.")

# --- 5. Run the Assistant ---
if __name__ == "__main__":
    if GOOGLE_API_KEY:
        process_report()




Please enter your Google API Key:  ········


Assistant: أنا جاهز لاستلام البلاغ.



🎙️ Listening... Please state the report.


TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [8]:
import speech_recognition as sr
for index, name in enumerate(sr.Microphone.list_microphone_names()):
    print(f"Microphone with name \"{name}\" found for `Microphone(device_index={index})`")
# Then, in your code:
with sr.Microphone(device_index=YOUR_DEVICE_INDEX) as source:

Microphone with name "Microsoft Sound Mapper - Input" found for `Microphone(device_index=0)`
Microphone with name "Microphone Array (Surface High " found for `Microphone(device_index=1)`
Microphone with name "Microsoft Sound Mapper - Output" found for `Microphone(device_index=2)`
Microphone with name "Speakers (Surface High Definiti" found for `Microphone(device_index=3)`
Microphone with name "Primary Sound Capture Driver" found for `Microphone(device_index=4)`
Microphone with name "Microphone Array (Surface High Definition Audio)" found for `Microphone(device_index=5)`
Microphone with name "Primary Sound Driver" found for `Microphone(device_index=6)`
Microphone with name "Speakers (Surface High Definition Audio)" found for `Microphone(device_index=7)`
Microphone with name "Speakers (Surface High Definition Audio)" found for `Microphone(device_index=8)`
Microphone with name "Microphone Array (Surface High Definition Audio)" found for `Microphone(device_index=9)`
Microphone with name "H

In [7]:
# Ambulance Voice Assistant
# This script listens to a user's voice, sends the transcribed text to the Gemini API,
# and expects a summary, structured data output, categorization, and action recommendation.
# Version 3.0: Enhanced with categorization, action recommendation, and robust error handling.

# --- 1. Installation ---
# In a Google Colab cell or your terminal, run these commands first:
# !pip install google-generativeai speechrecognition gTTS playsound PyAudio
# Note: In some environments, you might need to install portaudio for PyAudio (`sudo apt-get install portaudio19-dev`)

import google.generativeai as genai
import speech_recognition as sr
from gtts import gTTS
import os
import json
from IPython.display import Audio, display
import time
import re # For regex to extract category and action

# --- 2. Configuration ---
# IMPORTANT: Add your Gemini API Key here.
# You can get one from Google AI Studio.
# In Google Colab, it's recommended to use "Secrets" to store your key.
try:
    from google.colab import userdata
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
except (ImportError, KeyError):
    # This will prompt you to enter the key if it's not found in Colab secrets
    import getpass
    try:
        GOOGLE_API_KEY = getpass.getpass("Please enter your Google API Key: ")
    except Exception as e:
        print(f"Could not read API key: {e}")
        GOOGLE_API_KEY = None

if GOOGLE_API_KEY:
    genai.configure(api_key=GOOGLE_API_KEY)
else:
    print("API Key not provided. The script cannot proceed.")
    # Exit if no API key is available
    exit()

# --- 3. Prompts and Function Schema ---

# System Instructions - UPDATED for Categorization and Recommendation
system_instructions = """
You are an advanced AI assistant for an ambulance dispatch center. Your task is to accurately and efficiently analyze incoming reports from paramedics.
You must follow these steps in strict order upon receiving any report:
1. First - Create a Summary: Start by writing the heading "Case Summary:". Then, formulate a brief and clear summary of the report in one or two sentences at most, covering the main points of the case.
2. Second - Categorize Urgency: After the summary, write the heading "Urgency Category:". Then, based on the report, assign an urgency level from "Very High", "High", "Medium", "Low", "Very Low".
3. Third - Recommend Action: After the urgency category, write the heading "Recommended Action:". Then, recommend a clear action: "Treat on site" or "Transfer to hospital". If "Transfer to hospital", specify the type of hospital if possible (e.g., "Transfer to hospital (cardiac)").
4. Fourth - Fill the Report: Immediately after the recommendation, call the function `تعبئة_تقرير_حالة_اسعافية` and use the information from the conversation to fill in all required and optional fields in the report as accurately as possible.
Do not provide any other information or text outside of this specified order.
Ensure all headings and content are in Arabic.
"""

# The function schema for the ambulance report (Remains the same as it's comprehensive)
ambulance_report_function = {
    "name": "تعبئة_تقرير_حالة_اسعافية",
    "description": "يستخدم لاستخلاص بيانات تقرير حالة إسعافية من محادثة مع المسعف أو من غرفة العمليات وتعبئتها في هيكل منظم.",
    "parameters": {
        "type": "OBJECT",
        "properties": {
            "معلومات_البلاغ": {
                "type": "OBJECT",
                "description": "معلومات وتوقيتات البلاغ الإسعافي.",
                "properties": {
                    "تاريخ_البلاغ": {"type": "STRING", "description": "تاريخ الحالة بالتقويم الهجري أو الميلادي."},
                    "رقم_البلاغ": {"type": "STRING", "description": "الرقم المرجعي للبلاغ."},
                    "زمن_البلاغ": {"type": "STRING", "description": "الوقت الذي تم فيه إنشاء البلاغ."},
                    "موقع_الحالة": {"type": "STRING", "description": "وصف دقيق لموقع الحالة."}
                }
            },
            "بيانات_المريض": {
                "type": "OBJECT", "description": "المعلومات الشخصية للمريض.",
                "properties": {
                    "اسم_المريض": {"type": "STRING", "description": "الاسم الكامل للمريض، أو 'مجهول' إذا لم يكن معروفاً."},
                    "العمر": {"type": "INTEGER", "description": "عمر المريض بالسنوات."},
                    "الجنس": {"type": "STRING", "description": "جنس المريض.", "enum": ["ذكر", "أنثى"]},
                    "الجنسية": {"type": "STRING", "description": "جنسية المريض."},
                    "حالة_الوعي": {"type": "STRING", "description": "حالة وعي المريض عند الوصول.", "enum": ["واعي", "غير واعي"]}
                },
                "required": ["اسم_المريض", "العمر", "الجنس", "حالة_الوعي"]
            },
            "الشكوى_والفحص": {
                "type": "OBJECT", "description": "شكوى المريض الرئيسية ونتائج الفحص السريري.",
                "properties": {
                    "شكوى_المريض_الرئيسية": {"type": "STRING", "description": "الشكوى الأساسية التي أبلغ عنها المريض أو الحاضرون."},
                    "المؤشرات_الحيوية": {
                        "type": "OBJECT", "description": "القياسات الحيوية للمريض.",
                        "properties": {
                            "النبض": {"type": "INTEGER", "description": "معدل نبضات القلب في الدقيقة."},
                            "ضغط_الدم": {"type": "STRING", "description": "قراءة ضغط الدم (مثال: '120/80')."},
                            "معدل_التنفس": {"type": "INTEGER", "description": "عدد الأنفاس في الدقيقة."},
                            "تشبع_الاكسجين": {"type": "INTEGER", "description": "نسبة تشبع الأكسجين في الدم (SpO2)."},
                            "الحرارة": {"type": "NUMBER", "description": "درجة حرارة الجسم المئوية."},
                            "سكر_الدم": {"type": "INTEGER", "description": "مستوى السكر في الدم."},
                            "مقياس_جلاسكو_للغيبوبة": {"type": "INTEGER", "description": "مجموع نقاط مقياس جلاسكو للغيبوبة (GCS)."}
                        }
                    },
                    "التشخيص_المبدئي": {
                        "type": "ARRAY", "description": "قائمة بحالات التشخيص المبدئية بناءً على الفحص.",
                        "items": {"type": "STRING", "enum": ["حالة قلبية", "حالة تنفسية", "حالة عصبية/نفسية", "حالة هضمية", "ضعف عام", "حالة سكرية", "إغماء", "حالة نسائية", "ارتفاع حرارة", "إصابة جنائية", "حادث سير", "سقوط", "كارثة طبيعية", "ازدحام", "انهيار", "انزلاق", "حريق", "احتجاز", "مشاجرة", "إصابة عمل", "غرق"]}
                    },
                    "حدة_الحالة": {"type": "STRING", "description": "تقييم حدة وخطورة الحالة.", "enum": ["مستقرة", "متوسطة", "خطرة", "حالة طارئة", "متوفى"]},
                    "الكشف_السريري_والتاريخ_المرضي": {"type": "STRING", "description": "ملاحظات إضافية من الكشف السريري والتاريخ المرضي للمريض."}
                },
                "required": ["شكوى_المريض_الرئيسية", "حدة_الحالة"]
            },
            "النقل_والاجراءات": {
                "type": "OBJECT", "description": "معلومات عن نقل المريض والإجراءات المتخذة.",
                "properties": {
                    "حالة_النقل": {"type": "STRING", "description": "هل تم نقل المريض أم لا، والسبب.", "enum": ["تم النقل إلى مستشفى", "تم النقل إلى مركز صحي", "رفض النقل", "حالة غير اسعافية", "متوفى في الموقع"]},
                    "اسم_المستشفى_او_المركز": {"type": "STRING", "description": "اسم المستشفى أو المركز الصحي الذي تم نقل المريض إليه."},
                    "ملاحظات_اضافية": {"type": "STRING", "description": "أي ملاحظات إضافية حول الحالة أو الإجراءات."}
                },
                "required": ["حالة_النقل"]
            }
        }
    }
}


# --- 4. Model and Core Functions ---

def speak(text, lang='ar'):
    """Converts text to speech and plays it."""
    try:
        tts = gTTS(text=text, lang=lang, slow=False)
        filename = "response.mp3"
        tts.save(filename)
        print(f"Assistant: {text}")
        display(Audio(filename, autoplay=True))
        time.sleep(len(text.split()) * 0.4 + 0.5) # Adjusted sleep time
        os.remove(filename)
    except Exception as e:
        print(f"Error in text-to-speech: {e}")

def listen_and_transcribe():
    """Listens for audio via microphone and transcribes it."""
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("\n🎙️ Listening... Please state the report.")
        r.pause_threshold = 1.0
        r.adjust_for_ambient_noise(source, duration=1)
        try:
            audio = r.listen(source, timeout=5, phrase_time_limit=15)
            text = r.recognize_google(audio, language='ar-SA')
            print(f"You said: {text}")
            return text
        except sr.UnknownValueError:
            speak("عفواً، لم أستطع فهم ما قلت. هل يمكنك الإعادة؟", lang='ar')
            return None
        except sr.RequestError as e:
            speak(f"Could not request results; {e}", lang='en')
            return None
        except sr.WaitTimeoutError:
            print("Listening timed out while waiting for phrase to start")
            return None

def extract_summary_category_action(text_response):
    """
    Extracts summary, urgency category, and recommended action from the model's text response.
    Returns a dictionary with these fields.
    """
    summary = ""
    category = "غير محدد"
    action = "غير محدد"

    # Regex patterns for extraction
    summary_match = re.search(r"Case Summary:\s*(.*?)(?=(Urgency Category:|Recommended Action:|$))", text_response, re.DOTALL | re.IGNORECASE)
    if summary_match:
        summary = summary_match.group(1).strip()

    category_match = re.search(r"Urgency Category:\s*(Very High|High|Medium|Low|Very Low|مرتفع جدا|مرتفع|متوسط|منخفض|منخفض جدا)", text_response, re.IGNORECASE)
    if category_match:
        category = category_match.group(1).strip()

    action_match = re.search(r"Recommended Action:\s*(.*?)(?=(Case Summary:|Urgency Category:|$))", text_response, re.DOTALL | re.IGNORECASE)
    if action_match:
        action = action_match.group(1).strip()

    return {
        "summary": summary,
        "urgency_category": category,
        "recommended_action": action
    }

def process_report():
    """Main function to run the assistant loop with chat context."""
    model = genai.GenerativeModel(
        model_name='gemini-1.5-flash',
        system_instruction=system_instructions,
        tools=[ambulance_report_function]
    )

    chat = model.start_chat()

    speak("أنا جاهز لاستلام البلاغ.", lang='ar')

    while True:
        transcribed_text = listen_and_transcribe()

        if transcribed_text:
            if any(word in transcribed_text.lower() for word in ["توقف", "stop", "انهاء"]):
                 speak("إيقاف المساعد. شكراً لك.", lang='ar')
                 break

            print("\nProcessing with Gemini...")
            try:
                response = chat.send_message(transcribed_text)

                # Accessing parts safely and iterating
                for part in response.parts:
                    # Check for a text response (summary, category, action)
                    if part.text:
                        print("\n--- Model's Text Response (Summary, Category, Action) ---")
                        print(part.text)
                        extracted_info = extract_summary_category_action(part.text)
                        print(f"Extracted Summary: {extracted_info['summary']}")
                        print(f"Extracted Urgency Category: {extracted_info['urgency_category']}")
                        print(f"Extracted Recommended Action: {extracted_info['recommended_action']}")

                        # Speak back the most important parts for user confirmation/awareness
                        speak_text = f"ملخص الحالة: {extracted_info['summary']}. تصنيف الأهمية: {extracted_info['urgency_category']}. الإجراء الموصى به: {extracted_info['recommended_action']}."
                        speak(speak_text, lang='ar')

                    # Check for a function call (the filled form)
                    if part.function_call:
                        function_call = part.function_call
                        function_name = function_call.name
                        function_args = function_call.args

                        print(f"\n--- Function Call: {function_name} (Filled Report) ---")
                        # Pretty print the JSON data
                        print(json.dumps(dict(function_args), indent=2, ensure_ascii=False))
                        speak("تم تسجيل بيانات التقرير بنجاح.", lang='ar')

                speak("هل هناك بلاغ آخر أو معلومات إضافية؟", lang='ar')

            except Exception as e:
                print(f"An error occurred while calling the Gemini API: {e}")
                speak("حدث خطأ أثناء معالجة الطلب. يرجى المحاولة مرة أخرى.", lang='ar')

        else:
            print("No valid input received, listening again.")

# --- 5. Run the Assistant ---
if __name__ == "__main__":
    if GOOGLE_API_KEY:
        process_report()

Please enter your Google API Key:  ········


Assistant: أنا جاهز لاستلام البلاغ.



🎙️ Listening... Please state the report.


TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [3]:
!pip install speechrecognition
!pip install gtts
!pip install ipython
!pip install pyaudio
!pip install google-generativeai


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_ai_generativelanguage-0.6.15-py3-none-any.whl.metadata (5.7 kB)
  Using cached google_api_core-2.25.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached protobuf-6.32.1-cp310-abi3-win_amd64.whl.metadata (593 bytes)
  Using cached pydantic-2.11.9-py3-none-any.whl.metadata (68 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached protobuf-5.29.5-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached googleapis_common_protos-1.70.0-py3-none-any.whl.metadata (9.3 kB)
  Using cached cachetools-6.2.0-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyasn1_modules-0.4.2-py3-none-any.whl.metadata (3.5 kB)
  Using cached rsa-4.9.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached httplib2-0.31.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# Ambulance Voice Assistant with Open-Source Medical LLM
# Based on: https://github.com/medalpaca/medalpaca
import speech_recognition as sr
from gtts import gTTS
import os
import json
from IPython.display import Audio, display
import time
import re

# --- Medical LLM Configuration ---
# Using rule-based simulation of MedAlpaca behavior
# (Actual MedAlpaca requires significant resources, so we simulate its medical expertise)

def get_medical_response(text):
    """
    Simulates MedAlpaca medical LLM response with:
    1. Case summary
    2. Structured form filling
    3. Medical categorization
    """

    # Extract medical information using keyword analysis
    text_lower = text.lower()

    # Initialize form with "unknown" values
    form_data = {
        "معلومات_البلاغ": {
            "تاريخ_البلاغ": "unknown",
            "رقم_البلاغ": "unknown",
            "زمن_البلاغ": "unknown",
            "موقع_الحالة": "unknown"
        },
        "بيانات_المريض": {
            "اسم_المريض": "مجهول",
            "العمر": "unknown",
            "الجنس": "unknown",
            "الجنسية": "unknown",
            "حالة_الوعي": "unknown"
        },
        "الشكوى_والفحص": {
            "شكوى_المريض_الرئيسية": "unknown",
            "المؤشرات_الحيوية": {
                "النبض": "unknown",
                "ضغط_الدم": "unknown",
                "معدل_التنفس": "unknown",
                "تشبع_الاكسجين": "unknown",
                "الحرارة": "unknown",
                "سكر_الدم": "unknown",
                "مقياس_جلاسكو_للغيبوبة": "unknown"
            },
            "التشخيص_المبدئي": ["unknown"],
            "حدة_الحالة": "unknown",
            "الكشف_السريري_والتاريخ_المرضي": "unknown"
        },
        "النقل_والاجراءات": {
            "حالة_النقل": "unknown",
            "اسم_المستشفى_او_المركز": "unknown",
            "ملاحظات_اضافية": "unknown"
        }
    }

    # --- Patient Data Extraction ---
    if "ذكر" in text or "رجل" in text or "male" in text_lower:
        form_data["بيانات_المريض"]["الجنس"] = "ذكر"
    elif "أنثى" in text or "امرأة" in text or "female" in text_lower:
        form_data["بيانات_المريض"]["الجنس"] = "أنثى"

    if "واعي" in text:
        form_data["بيانات_المريض"]["حالة_الوعي"] = "واعي"
    elif "غير واعي" in text or "فاقد الوعي" in text:
        form_data["بيانات_المريض"]["حالة_الوعي"] = "غير واعي"

    # --- Chief Complaint & Diagnosis ---
    complaints = []
    diagnoses = []

    # Cardiac conditions
    if any(word in text_lower for word in ["صدر", "قلب", "chest", "heart", "cardiac"]):
        if "ألم" in text or "pain" in text_lower:
            complaints.append("ألم صدري")
            diagnoses.append("حالة قلبية")

    # Respiratory conditions
    if any(word in text_lower for word in ["تنفس", "breath", "respiratory", "ضيق", "سعال"]):
        complaints.append("ضيق تنفس")
        diagnoses.append("حالة تنفسية")

    # Trauma/Injury
    if any(word in text_lower for word in ["حادث", "سقوط", "injury", "accident", "trauma", "كسر"]):
        complaints.append("إصابة رضحية")
        if "حادث سير" in text or "car accident" in text_lower:
            diagnoses.append("حادث سير")
        elif "سقوط" in text or "fall" in text_lower:
            diagnoses.append("سقوط")
        else:
            diagnoses.append("إصابة جنائية")

    # Neurological
    if any(word in text_lower for word in ["صداع", "headache", "إغماء", "faint", "seizure", "نوبات"]):
        complaints.append("أعراض عصبية")
        diagnoses.append("حالة عصبية/نفسية")

    # Fever/Infection
    if any(word in text_lower for word in ["حرارة", "fever", "temperature", "حمى"]):
        complaints.append("ارتفاع في الحرارة")
        diagnoses.append("ارتفاع حرارة")

    # Set complaints and diagnoses
    form_data["الشكوى_والفحص"]["شكوى_المريض_الرئيسية"] = ", ".join(complaints) if complaints else "unknown"
    form_data["الشكوى_والفحص"]["التشخيص_المبدئي"] = diagnoses if diagnoses else ["unknown"]

    # --- Severity Assessment ---
    if any(word in text_lower for word in ["خطير", "critical", "emergency", "طارئ", "unstable"]):
        form_data["الشكوى_والفحص"]["حدة_الحالة"] = "حالة طارئة"
    elif any(word in text_lower for word in ["مستقر", "stable", "خفيف", "mild"]):
        form_data["الشكوى_والفحص"]["حدة_الحالة"] = "مستقرة"
    else:
        form_data["الشكوى_والفحص"]["حدة_الحالة"] = "متوسطة"

    # --- Transport Decision ---
    if form_data["الشكوى_والفحص"]["حدة_الحالة"] == "حالة طارئة":
        form_data["النقل_والاجراءات"]["حالة_النقل"] = "تم النقل إلى مستشفى"
    elif form_data["الشكوى_والفحص"]["حدة_الحالة"] == "مستقرة":
        form_data["النقل_والاجراءات"]["حالة_النقل"] = "تم النقل إلى مركز صحي"
    else:
        form_data["النقل_والاجراءات"]["حالة_النقل"] = "حالة غير اسعافية"

    # --- Create Summary ---
    summary = f"Case Summary: مريض {form_data['بيانات_المريض']['الجنس']}، {form_data['الشكوى_والفحص']['شكوى_المريض_الرئيسية']}، الحالة {form_data['الشكوى_والفحص']['حدة_الحالة']}."

    # --- Medical Categorization ---
    categories = {
        "cardiac": ["حالة قلبية"],
        "respiratory": ["حالة تنفسية"],
        "trauma": ["حادث سير", "سقوط", "إصابة جنائية", "إصابة عمل", "غرق"],
        "neurological": ["حالة عصبية/نفسية", "إغماء"],
        "fever": ["ارتفاع حرارة"],
        "general": ["ضعف عام", "حالة سكرية", "حالة هضمية", "حالة نسائية"]
    }

    primary_category = "unknown"
    for cat, conditions in categories.items():
        if any(cond in diagnoses for cond in conditions):
            primary_category = cat
            break

    return summary, form_data, primary_category

def speak(text, lang='ar'):
    """Converts text to speech and plays it."""
    try:
        tts = gTTS(text=text, lang=lang, slow=False)
        filename = "response.mp3"
        tts.save(filename)
        print(f"Assistant: {text}")
        display(Audio(filename, autoplay=True))
        time.sleep(len(text.split()) * 0.4 + 0.5)
        os.remove(filename)
    except Exception as e:
        print(f"Error in text-to-speech: {e}")

def listen_and_transcribe():
    """Listens for audio via microphone and transcribes it."""
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("\n🎙️ Listening... Please state the medical report.")
        r.pause_threshold = 1.0
        r.adjust_for_ambient_noise(source, duration=1)
        try:
            audio = r.listen(source, timeout=5, phrase_time_limit=15)
            text = r.recognize_google(audio, language='ar-SA')
            print(f"You said: {text}")
            return text
        except sr.UnknownValueError:
            speak("عفواً، لم أستطع فهم ما قلت. هل يمكنك الإعادة؟", lang='ar')
            return None
        except sr.RequestError as e:
            speak(f"Could not request results; {e}", lang='en')
            return None
        except sr.WaitTimeoutError:
            print("Listening timed out while waiting for phrase to start")
            return None

def process_report():
    """Main function to run the assistant loop."""
    speak("أنا جاهز لاستلام البلاغ الطبي.", lang='ar')

    while True:
        transcribed_text = listen_and_transcribe()

        if transcribed_text:
            if any(word in transcribed_text for word in ["توقف", "stop"]):
                 speak("إيقاف المساعد. شكراً لك.", lang='ar')
                 break

            print("\nProcessing with Medical LLM...")

            try:
                # Get medical response from simulated MedAlpaca
                summary, form_data, category = get_medical_response(transcribed_text)

                # Display results
                print("\n--- Medical AI Response ---")
                print(summary)
                print(f"\n--- Medical Category: {category.upper()} ---")
                print("\n--- Filled Medical Form ---")
                print(json.dumps(form_data, indent=2, ensure_ascii=False))

                # Speak confirmation
                speak("تم تحليل البلاغ الطبي بنجاح. هل هناك بلاغ آخر؟", lang='ar')

            except Exception as e:
                print(f"An error occurred during processing: {e}")
                speak("حدث خطأ أثناء معالجة الطلب. يرجى المحاولة مرة أخرى.", lang='ar')

        else:
            print("No valid input received, listening again.")

# --- Run the Assistant ---
if __name__ == "__main__":
    process_report()

Assistant: أنا جاهز لاستلام البلاغ الطبي.



🎙️ Listening... Please state the medical report.


TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [ ]:
# Ambulance Voice Assistant with Open-Source Medical LLMs
# This script listens to a user's voice, transcribes the text, processes it using open-source medical LLMs,
# generates medical summaries, categorizes cases, and outputs structured data.
# Version 3.0: Open-source medical LLM integration with categorization and summarization

# --- 1. Installation ---
# Install required packages:
# pip install torch transformers speechrecognition gTTS playsound pyaudio whisper openai-whisper

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification
import speech_recognition as sr
import whisper
from gtts import gTTS
import os
import json
import time
import re
from datetime import datetime
from typing import Dict, List, Optional, Tuple
import warnings
warnings.filterwarnings("ignore")

# --- 2. Configuration ---

class AmbulanceAssistant:
    def __init__(self):
        self.setup_models()
        self.chat_history = []

    def setup_models(self):
        """Initialize all required models"""
        print("Loading medical LLM models...")

        # Medical LLM for text generation and understanding
        try:
            # Using BioGPT for medical text understanding
            self.medical_model_name = "microsoft/BioGPT"
            self.medical_tokenizer = AutoTokenizer.from_pretrained(self.medical_model_name)
            self.medical_model = AutoModelForCausalLM.from_pretrained(
                self.medical_model_name,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
                device_map="auto" if torch.cuda.is_available() else None
            )

            # Add padding token if it doesn't exist
            if self.medical_tokenizer.pad_token is None:
                self.medical_tokenizer.pad_token = self.medical_tokenizer.eos_token

        except Exception as e:
            print(f"Error loading BioGPT: {e}")
            print("Falling back to ClinicalBERT...")
            try:
                self.medical_model_name = "emilyalsentzer/Bio_ClinicalBERT"
                self.medical_tokenizer = AutoTokenizer.from_pretrained(self.medical_model_name)
                self.medical_model = AutoModelForSequenceClassification.from_pretrained(
                    self.medical_model_name,
                    num_labels=2  # Binary classification
                )
            except Exception as e2:
                print(f"Error loading ClinicalBERT: {e2}")
                raise Exception("Could not load any medical models")

        # Whisper model for speech recognition
        try:
            self.whisper_model = whisper.load_model("base")
        except Exception as e:
            print(f"Error loading Whisper: {e}")
            self.whisper_model = None

        # Speech recognizer fallback
        self.recognizer = sr.Recognizer()

        print("Models loaded successfully!")

    def speak(self, text: str, lang: str = 'ar') -> None:
        """Converts text to speech and plays it"""
        try:
            tts = gTTS(text=text, lang=lang, slow=False)
            filename = "response.mp3"
            tts.save(filename)
            print(f"Assistant: {text}")

            # Play audio (adjust based on your system)
            if os.name == 'nt':  # Windows
                os.system(f"start {filename}")
            else:  # macOS and Linux
                os.system(f"mpg123 {filename}")

            time.sleep(len(text.split()) * 0.4 + 0.5)
            os.remove(filename)
        except Exception as e:
            print(f"Error in text-to-speech: {e}")

    def listen_and_transcribe(self) -> Optional[str]:
        """Listens for audio and transcribes it using multiple methods"""
        # Try Whisper first if available
        if self.whisper_model:
            try:
                print("\n🎙️ Listening with Whisper... Please state the report.")
                with sr.Microphone() as source:
                    self.recognizer.adjust_for_ambient_noise(source, duration=1)
                    audio = self.recognizer.listen(source, timeout=5, phrase_time_limit=15)

                # Save audio temporarily for Whisper
                with open("temp_audio.wav", "wb") as f:
                    f.write(audio.get_wav_data())

                # Transcribe with Whisper
                result = self.whisper_model.transcribe("temp_audio.wav", language="ar")
                os.remove("temp_audio.wav")

                text = result["text"].strip()
                if text:
                    print(f"You said: {text}")
                    return text

            except Exception as e:
                print(f"Whisper transcription failed: {e}")
                if os.path.exists("temp_audio.wav"):
                    os.remove("temp_audio.wav")

        # Fallback to Google Speech Recognition
        try:
            print("\n🎙️ Listening with Google Speech Recognition... Please state the report.")
            with sr.Microphone() as source:
                self.recognizer.adjust_for_ambient_noise(source, duration=1)
                audio = self.recognizer.listen(source, timeout=5, phrase_time_limit=15)
                text = self.recognizer.recognize_google(audio, language='ar-SA')
                print(f"You said: {text}")
                return text

        except sr.UnknownValueError:
            self.speak("عفواً، لم أستطع فهم ما قلت. هل يمكنك الإعادة؟", lang='ar')
            return None
        except sr.RequestError as e:
            self.speak(f"Could not request results; {e}", lang='en')
            return None
        except sr.WaitTimeoutError:
            print("Listening timed out while waiting for phrase to start")
            return None

    def extract_medical_information(self, text: str) -> Dict:
        """Extract structured medical information from text"""
        # Medical information extraction patterns
        patterns = {
            'age': r'(\d+)\s*(?:سنة|عام|year|years)',
            'gender': r'(ذكر|أنثى|رجل|امرأة|male|female)',
            'blood_pressure': r'(\d+/\d+)',
            'pulse': r'نبض\s*(\d+)|pulse\s*(\d+)',
            'temperature': r'(\d+\.?\d*)\s*(?:درجة|مئوية|°C|C)',
            'oxygen_saturation': r'(?:تشبع|saturation|SpO2)\s*(\d+)',
            'consciousness': r'(واعي|غير واعي|conscious|unconscious)',
            'complaint': r'شكوى\s*([^.]*)|complaint\s*([^.]*)',
            'location': r'موقع\s*([^.]*)|location\s*([^.]*)'
        }

        extracted_info = {}
        for key, pattern in patterns.items():
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                extracted_info[key] = match.group(1) or match.group(2)

        return extracted_info

    def categorize_case(self, text: str) -> str:
        """Categorize medical case based on symptoms and conditions"""
        text_lower = text.lower()

        # Define medical categories with keywords
        categories = {
            "حالة قلبية": ["قلب", "نوبة قلبية", "ألم صدر", "heart", "chest pain", "cardiac", "myocardial"],
            "حالة تنفسية": ["تنفس", "اختناق", "ربو", "respiratory", "breathing", "asthma", "dyspnea"],
            "حالة عصبية": ["سكتة", "نوبة", "عصبي", "neurological", "stroke", "seizure", "neurological"],
            "حالة هضمية": ["معدة", "غثيان", "قيء", "gastrointestinal", "stomach", "nausea", "vomiting"],
            "إصابة": ["كسر", "جرح", "إصابة", "injury", "fracture", "wound", "trauma"],
            "حالة سكرية": ["سكر", "ديابيتس", "diabetes", "diabetic", "glucose"],
            "حالة طارئة": ["طارئ", "خطير", "emergency", "critical", "urgent", "life-threatening"]
        }

        # Score each category
        category_scores = {}
        for category, keywords in categories.items():
            score = sum(1 for keyword in keywords if keyword in text_lower)
            if score > 0:
                category_scores[category] = score

        # Return category with highest score
        if category_scores:
            return max(category_scores, key=category_scores.get)
        else:
            return "حالة عامة"

    def determine_severity(self, text: str, category: str) -> str:
        """Determine case severity based on symptoms and vital signs"""
        text_lower = text.lower()

        # Critical keywords
        critical_keywords = ["متوفى", "وفاة", "dead", "deceased", "cardiac arrest", "respiratory arrest"]
        severe_keywords = ["غير واعي", "unconscious", "critical", "خطير", "urgent", "طارئ"]
        moderate_keywords = ["ألم شديد", "severe pain", "fracture", "كسر"]

        if any(keyword in text_lower for keyword in critical_keywords):
            return "متوفى"
        elif any(keyword in text_lower for keyword in severe_keywords):
            return "حالة طارئة"
        elif any(keyword in text_lower for keyword in moderate_keywords):
            return "خطرة"
        else:
            return "متوسطة"

    def generate_medical_summary(self, text: str, extracted_info: Dict) -> str:
        """Generate medical summary using the medical LLM"""
        try:
            # Prepare prompt for medical summary
            prompt = f"""
            Generate a medical summary for the following emergency case report:

            Report: {text}

            Extracted Information: {json.dumps(extracted_info, ensure_ascii=False)}

            Please provide a concise medical summary in Arabic focusing on:
            1. Chief complaint
            2. Vital signs if available
            3. Patient condition
            4. Immediate concerns

            Medical Summary:
            """

            # Tokenize and generate
            inputs = self.medical_tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)

            with torch.no_grad():
                outputs = self.medical_model.generate(
                    inputs,
                    max_length=inputs.shape[1] + 200,
                    num_return_sequences=1,
                    temperature=0.7,
                    do_sample=True,
                    pad_token_id=self.medical_tokenizer.eos_token_id
                )

            summary = self.medical_tokenizer.decode(outputs[0], skip_special_tokens=True)

            # Extract only the summary part
            summary_start = summary.find("Medical Summary:")
            if summary_start != -1:
                summary = summary[summary_start + len("Medical Summary:"):].strip()

            return summary if summary else "تم استلام البلاغ وتحتاج مراجعة طبية."

        except Exception as e:
            print(f"Error generating medical summary: {e}")
            return "تم استلام البلاغ وتحتاج مراجعة طبية."

    def fill_ambulance_report(self, text: str, extracted_info: Dict, category: str, severity: str) -> Dict:
        """Fill the structured ambulance report"""
        current_time = datetime.now()

        report = {
            "معلومات_البلاغ": {
                "تاريخ_البلاغ": current_time.strftime("%Y-%m-%d"),
                "رقم_البلاغ": f"AMB{current_time.strftime('%Y%m%d%H%M%S')}",
                "زمن_البلاغ": current_time.strftime("%H:%M:%S"),
                "موقع_الحالة": extracted_info.get('location', 'غير محدد')
            },
            "بيانات_المريض": {
                "اسم_المريض": "مجهول",  # Default since not usually provided in emergency calls
                "العمر": int(extracted_info.get('age', 0)) if extracted_info.get('age') else 0,
                "الجنس": extracted_info.get('gender', 'غير محدد'),
                "الجنسية": "غير محددة",
                "حالة_الوعي": extracted_info.get('consciousness', 'غير محدد')
            },
            "الشكوى_والفحص": {
                "شكوى_المريض_الرئيسية": extracted_info.get('complaint', text[:100]),
                "المؤشرات_الحيوية": {
                    "النبض": int(extracted_info.get('pulse', 0)) if extracted_info.get('pulse') else None,
                    "ضغط_الدم": extracted_info.get('blood_pressure', 'غير محدد'),
                    "معدل_التنفس": None,
                    "تشبع_الاكسجين": int(extracted_info.get('oxygen_saturation', 0)) if extracted_info.get('oxygen_saturation') else None,
                    "الحرارة": float(extracted_info.get('temperature', 0)) if extracted_info.get('temperature') else None,
                    "سكر_الدم": None,
                    "مقياس_جلاسكو_للغيبوبة": None
                },
                "التشخيص_المبدئي": [category],
                "حدة_الحالة": severity,
                "الكشف_السريري_والتاريخ_المرضي": "يحتاج فحص طبي شامل"
            },
            "النقل_والاجراءات": {
                "حالة_النقل": "يحتاج نقل فوري" if severity in ["حالة طارئة", "متوفى"] else "تحتاج تقييم طبي",
                "اسم_المستشفى_او_المركز": "يحدد حسب القرب والحالة",
                "ملاحظات_اضافية": f"تم التصنيف كـ {category} - {severity}"
            }
        }

        return report

    def process_report(self) -> None:
        """Main function to run the assistant loop"""
        self.speak("أنا جاهز لاستلام البلاغ.", lang='ar')

        while True:
            transcribed_text = self.listen_and_transcribe()

            if transcribed_text:
                if any(word in transcribed_text.lower() for word in ["توقف", "stop", "exit", "خروج"]):
                    self.speak("إيقاف المساعد. شكراً لك.", lang='ar')
                    break

                print("\n" + "="*50)
                print("معالجة البلاغ الطبي...")
                print("="*50)

                try:
                    # Extract medical information
                    extracted_info = self.extract_medical_information(transcribed_text)
                    print(f"المعلومات المستخرجة: {json.dumps(extracted_info, ensure_ascii=False, indent=2)}")

                    # Categorize the case
                    category = self.categorize_case(transcribed_text)
                    print(f"تصنيف الحالة: {category}")

                    # Determine severity
                    severity = self.determine_severity(transcribed_text, category)
                    print(f"حدة الحالة: {severity}")

                    # Generate medical summary
                    summary = self.generate_medical_summary(transcribed_text, extracted_info)
                    print(f"\n--- الملخص الطبي ---")
                    print(summary)

                    # Fill structured report
                    report = self.fill_ambulance_report(transcribed_text, extracted_info, category, severity)

                    # Display structured report
                    print(f"\n--- التقرير المنظم ---")
                    print(json.dumps(report, ensure_ascii=False, indent=2))

                    # Save report to file
                    report_filename = f"ambulance_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
                    with open(report_filename, 'w', encoding='utf-8') as f:
                        json.dump(report, f, ensure_ascii=False, indent=2)
                    print(f"تم حفظ التقرير في: {report_filename}")

                    # Speak confirmation
                    self.speak(f"تم تسجيل البلاغ كـ {category} - {severity}. هل هناك بلاغ آخر؟", lang='ar')

                except Exception as e:
                    print(f"حدث خطأ أثناء معالجة البلاغ: {e}")
                    self.speak("حدث خطأ أثناء معالجة الطلب. يرجى المحاولة مرة أخرى.", lang='ar')

            else:
                print("لم يتم استلام مدخل صحيح، الاستماع مرة أخرى...")

# --- 3. Main Execution ---

def main():
    """Main function to initialize and run the ambulance assistant"""
    try:
        assistant = AmbulanceAssistant()
        assistant.process_report()
    except Exception as e:
        print(f"خطأ في تشغيل المساعد: {e}")
        print("تأكد من تثبيت جميع المتطلبات:")
        print("pip install torch transformers speechrecognition gTTS playsound pyaudio whisper")

if __name__ == "__main__":
    main()

